In [1]:
from edahelper import *
from bs4 import BeautifulSoup
from datetime import timezone, datetime
import networkx as nx

In [2]:
df = pd.read_pickle("../Data/comment_forests_temp_cleaned.pkl")
df

,created_datetime_utc,score,selftext,controversiality,depth,total_awards_received,likes,saved,id,author,all_awardings,stickied,body_html,author_fullname,name,_replies,parent_id,_submission,score_hidden
0,2020-01-29 20:21:47,1293,"Jesus Christ Almighty, how much cocaine did yo...",0,0,1,None,False,ffxoius,Sandvicheater,"[{'giver_coin_reward': None, 'subreddit_id': N...",False,"<div class=""md""><p>Jesus Christ Almighty, how ...",t2_441uw,t1_ffxoius,"(ffxpnet, fgj6ng1, ffyqzws, ffy5r1q, ffyg5q8, ...",t3_evs9eq,evs9eq,False
1,2020-01-29 21:00:32,395,I love this subreddit but it sometimes hurts s...,0,0,0,None,False,ffxsnst,Rtzizle,[],False,"<div class=""md""><p>I love this subreddit but i...",t2_1wz765kk,t1_ffxsnst,"(ffy8m86, fgghejz, fghrh2m, fgiosy4, fgiuf2n, ...",t3_evs9eq,evs9eq,False
2,2020-01-29 20:29:24,524,YES! NOW THIS IS A FUCKING YOLO!\n,0,0,0,None,False,ffxpcdf,karlaxel2,[],False,"<div class=""md""><p>YES! NOW THIS IS A FUCKING ...",t2_2onx9srs,t1_ffxpcdf,"(ffxrnkg, fg0jw0a)",t3_evs9eq,evs9eq,False
3,2020-01-29 21:41:11,241,According to the tastyworks trading platform t...,0,0,0,None,False,ffxwznv,kickeast,[],False,"<div class=""md""><p>According to the tastyworks...",t2_4o9dun45,t1_ffxwznv,"(ffy3i5f, ffyb4nm, ffy557b, ffy88km, fgq78gp, ...",t3_evs9eq,evs9eq,False
4,2020-01-29 20:26:33,447,"This can't be real. Nobody is this silly, righ...",0,0,0,None,False,ffxp1kn,stormwillpass,[],False,"<div class=""md""><p>This can&#39;t be real. Nob...",t2_zsjqy,t1_ffxp1kn,"(ffxt4ue, ffxs2jw, fgg5iai, ffyhfus, fgh7qq5, ...",t3_evs9eq,evs9eq,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143022,2021-05-04 20:00:52,1,Thank you\n,0,8,0,None,False,gwxwlxo,Soylto,[],False,"<div class=""md""><p>Thank you</p>\n</div>",t2_4jrq562v,t1_gwxwlxo,(),t1_gws1bq7,n3rb5l,False
143023,2021-05-03 15:43:05,-2,That's like...your opinion man\n,0,8,0,None,False,gwsm21c,TagMeAJerk,[],False,"<div class=""md""><p>That&#39;s like...your opin...",t2_lya07,t1_gwsm21c,(),t1_gws1bq7,n3rb5l,False
143024,2021-05-03 15:41:42,-1,"\nIn the current implementation, paper hands ....",0,8,0,None,False,gwsluv7,TagMeAJerk,[],False,"<div class=""md""><blockquote>\n<p>In the curren...",t2_lya07,t1_gwsluv7,(),t1_gwrwy24,n3rb5l,False
143025,2021-05-04 13:36:29,2,Exactly. I am not quit at the crossover momen...,0,8,0,None,False,gwwl7no,Pinknailzz69,[],False,"<div class=""md""><p>Exactly. I am not quit at ...",t2_66065hcx,t1_gwwl7no,(),t1_gwwht23,n3rb5l,False


## Social Network Graph


In [10]:
authors = set ( [x.name for x in df.author if x != None])

In [47]:
dfnamed[['author']]

,author
name,
t1_ffxoius,Sandvicheater
t1_ffxsnst,Rtzizle
t1_ffxpcdf,karlaxel2
t1_ffxwznv,kickeast
t1_ffxp1kn,stormwillpass
...,...
t1_gwxwlxo,Soylto
t1_gwsm21c,TagMeAJerk
t1_gwsluv7,TagMeAJerk


In [14]:
dfnamed = df.set_index('name')

In [54]:
df_wparent_names = dfnamed.merge( dfnamed[['author']], how = 'left', right_index = True, left_on = 'parent_id', suffixes = ('', '_parent'))


In [55]:
df_cleaned = df_wparent_names[['author', 'depth', 'score', 'author_parent']]

In [56]:
df_cleaned

,author,depth,score,author_parent
name,,,,
t1_ffxoius,Sandvicheater,0,1293,NaN
t1_ffxsnst,Rtzizle,0,395,NaN
t1_ffxpcdf,karlaxel2,0,524,NaN
t1_ffxwznv,kickeast,0,241,NaN
t1_ffxp1kn,stormwillpass,0,447,NaN
...,...,...,...,...
t1_gwxwlxo,Soylto,8,1,OPINION_IS_UNPOPULAR
t1_gwsm21c,TagMeAJerk,8,-2,OPINION_IS_UNPOPULAR
t1_gwsluv7,TagMeAJerk,8,-1,Agile-Hotel3809


In [57]:
social_network = nx.Graph()

In [59]:
for author in df_cleaned:
    social_network.add_node(author)

In [62]:
for index, row in df_cleaned.iterrows():
    if row.depth != 0:
        social_network.add_edge(row.author, row.author_parent)
        social_network.edges[ (row.author, row.author_parent)]['score'] = 0

In [63]:
for index, row in df_cleaned.iterrows():
    if row.depth != 0:
        social_network.edges[ (row.author, row.author_parent)]['score']  += row.score

In [64]:
comps = list(nx.connected_components(social_network))

In [65]:
pd.Series([len(x) for x in comps]).value_counts()

2        671
3        121
1         48
4         19
5         11
6          5
7          3
8          1
9          1
41450      1
dtype: int64

The isolated nodes are top level comments with no replies to them.

In [69]:
giant_component_set = [x for x in comps if len(x) == 41450][0]
giant_component = nx.subgraph(social_network, giant_component_set)

In [70]:
giant_component

In [77]:
for e in giant_component.edges:
    if giant_component.edges[e]['score'] < 0:
        print(giant_component.edges[e]['score'], e)

-3 (None, Redditor(name='lurkerCPA'))
-1 (None, Redditor(name='zdrmju321'))
-1 (None, Redditor(name='Hold_onto_yer_butts'))
-1 (None, Redditor(name='HAPPY__TECHNOLOGY'))
-6 (None, Redditor(name='Not_Sarkastic'))
-13 (None, Redditor(name='whereismikehawk'))
-1 (None, Redditor(name='ObsiArmyBest'))
-1 (None, Redditor(name='OWO-FurryPornAlt-OWO'))
-6 (None, Redditor(name='theetruscans'))
-9 (None, Redditor(name='Darck47'))
-6 (None, Redditor(name='Julez_Jay'))
-11 (None, Redditor(name='Kushbogga'))
-10 (None, Redditor(name='Tecashine'))
-1 (None, Redditor(name='arctic_bull'))
-4 (None, Redditor(name='moneymayhem'))
-1 (None, Redditor(name='KingsConsent'))
-1 (None, Redditor(name='PTRBoyz'))
-1 (None, Redditor(name='iN-Couchman'))
-1 (None, Redditor(name='fufm'))
-6 (None, Redditor(name='Crime_Dawg'))
-8 (None, Redditor(name='m1sta'))
-18 (None, Redditor(name='columbo928s4'))
-3 (None, Redditor(name='Deraneous'))
-91 (None, Redditor(name='Truth_I_Fear'))
-12 (None, Redditor(name='Arftacula

-3 (Redditor(name='Wowowiwa69'), Redditor(name='NarutoRunner'))
-1 (Redditor(name='mrbillsonsdad'), Redditor(name='heyheyfucktoday'))
-1 (Redditor(name='mrbillsonsdad'), Redditor(name='meowmixeree'))
-5 (Redditor(name='j_aurelius123'), Redditor(name='Lau_lau'))
-1 (Redditor(name='Sabilok'), Redditor(name='demotrek'))
-13 (Redditor(name='Tecashine'), Redditor(name='MyDadIsRacist'))
-1 (Redditor(name='Tecashine'), Redditor(name='farlack'))
-1 (Redditor(name='Tecashine'), Redditor(name='LSUsparky'))
-2 (Redditor(name='Tecashine'), Redditor(name='ItsFuckingScience'))
-2 (Redditor(name='OutOfBananaException'), Redditor(name='Gausauce'))
-2 (Redditor(name='Echo609'), Redditor(name='dnr41418'))
-3 (Redditor(name='nastystacks'), Redditor(name='Jerry-Can111'))
-3 (Redditor(name='gravityCaffeStocks'), Redditor(name='chamsticks'))
-9 (Redditor(name='feelin_cheesy'), Redditor(name='bebb69'))
-1 (Redditor(name='eboy4hire'), Redditor(name='the_dark_side'))
-4 (Redditor(name='BumbleFuckDuck'), Reddit

-1 (Redditor(name='pangolinolin'), Redditor(name='yogamatpat'))
-1 (Redditor(name='anthro28'), Redditor(name='SuppressedAvarice'))
-2 (Redditor(name='tianavitoli'), Redditor(name='thanosbutthole21'))
-15 (Redditor(name='PeakLL'), Redditor(name='YouWantSumFuck99'))
-50 (Redditor(name='nomadicwonder'), Redditor(name='Dantheman2247'))
-4 (Redditor(name='CorrosiveRose'), Redditor(name='marsinfurs'))
-1 (Redditor(name='absolutcity'), Redditor(name='lbryant1985'))
-1 (Redditor(name='ArtigoQ'), Redditor(name='Jestokost'))
-1 (Redditor(name='Say_no_to_doritos'), Redditor(name='stuntycunty'))
-3 (Redditor(name='Meatpopsicledream'), Redditor(name='exemplariasuntomni'))
-1 (Redditor(name='justherefercomments'), Redditor(name='coopers_recorder'))
-14 (Redditor(name='ta6514'), Redditor(name='GooieGui'))
-2 (Redditor(name='krushdavis247'), Redditor(name='havestronaut'))
-13 (Redditor(name='CiderDad'), Redditor(name='TheTurtler31'))
-3 (Redditor(name='neuron-'), Redditor(name='JohnnyMnemo'))
-3 (Redd

-1 (Redditor(name='hofferd78'), Redditor(name='snowandsorrow'))
-2 (Redditor(name='KAT-PWR'), Redditor(name='tomsmash1990'))
-2 (Redditor(name='i_sigh_less'), Redditor(name='crazyaznrobot'))
-5 (Redditor(name='PM_Me_Accounts_rGay'), Redditor(name='tragicb0t'))
-9 (Redditor(name='mjr2015'), Redditor(name='BitOfDifference'))
-18 (Redditor(name='mjr2015'), Redditor(name='fossdeep'))
-1 (Redditor(name='oli735'), Redditor(name='scottik187'))
-36 (Redditor(name='unnecessarychaos'), Redditor(name='Stan_Laurel1'))
-6 (Redditor(name='yahhhmoney'), Redditor(name='screamingzen'))
-10 (Redditor(name='apoliticalinactivist'), Redditor(name='BackgroundSearch30'))
-22 (Redditor(name='corgi_on_a_treadmill'), Redditor(name='Samycopter'))
-64 (Redditor(name='corgi_on_a_treadmill'), Redditor(name='zodar'))
-8 (Redditor(name='moutonbleu'), Redditor(name='lordpanda'))
-10 (Redditor(name='MasterKongQiu'), Redditor(name='AndrewLBailey'))
-1 (Redditor(name='flichter1'), Redditor(name='thegoodlucifer'))
-2 (Red

-35 (Redditor(name='DTX9218'), Redditor(name='OopsIPoodMyPants'))
-8 (Redditor(name='SunwindPC'), Redditor(name='Chicagorides'))
-4 (Redditor(name='toxic_masculinity27'), Redditor(name='unloud'))
-14 (Redditor(name='SuperValde'), Redditor(name='elyesisou'))
-5 (Redditor(name='zimmah'), Redditor(name='Lancaster61'))
-13 (Redditor(name='crzaznboi'), Redditor(name='Jeremy4pres'))
-16 (Redditor(name='bigjungus11'), Redditor(name='Chemical_Yoghurt6199'))
-1 (Redditor(name='sirgentrification'), Redditor(name='MrBeastTheT'))
-6 (Redditor(name='Joammo'), Redditor(name='SuddenStand'))
-2 (Redditor(name='D4ltaOne'), Redditor(name='Handsomedork_'))
-3 (Redditor(name='tradermike0429'), Redditor(name='Cheap_Confidence_657'))
-3 (Redditor(name='koreanjc'), Redditor(name='highcl1ff'))
-2 (Redditor(name='ForensicPaints'), Redditor(name='--X0X0--'))
-7 (Redditor(name='GravelGrinder07'), Redditor(name='IonBlade'))
-5 (Redditor(name='Notall_Knowledge'), Redditor(name='semitope'))
-1 (Redditor(name='Yuval

-7 (Redditor(name='ComfortMailbox'), Redditor(name='FrostyTemps'))
-23 (Redditor(name='Drewfus_'), Redditor(name='ark_mod'))
-3 (Redditor(name='Drewfus_'), Redditor(name='Locha6'))
-3 (Redditor(name='Drewfus_'), Redditor(name='joeyjoejoe314'))
-2 (Redditor(name='PursuitOfThis'), Redditor(name='crawwler'))
-4 (Redditor(name='crawwler'), Redditor(name='chingwoowang'))
-4 (Redditor(name='Futureib'), Redditor(name='Leandrir'))
-3 (Redditor(name='SweetnSour_DimSum'), Redditor(name='Thelandofthereal'))
-1 (Redditor(name='ark_mod'), Redditor(name='TheEvilWhiteMan'))
-1 (Redditor(name='ark_mod'), Redditor(name='PeddyCash'))
-3 (Redditor(name='ark_mod'), Redditor(name='_hitman11'))
-50 (Redditor(name='ark_mod'), Redditor(name='ark_mod'))
-23 (Redditor(name='ark_mod'), Redditor(name='Mvpeh'))
-1 (Redditor(name='MothAliens'), Redditor(name='chriswinsss'))
-1 (Redditor(name='MothAliens'), Redditor(name='Robinthekiid'))
-4 (Redditor(name='Sismal_Dystem'), Redditor(name='No_Tangerine5702'))
-1 (Redd

-1 (Redditor(name='cstrife007'), Redditor(name='Asking4Afren'))
-1 (Redditor(name='NefariousnessNoose'), Redditor(name='ThisIsWhoIAm78'))
-29 (Redditor(name='boatsnhoes801'), Redditor(name='Recreant57'))
-2 (Redditor(name='TheRedRebel4'), Redditor(name='Desalzes_'))
-18 (Redditor(name='TheRedRebel4'), Redditor(name='Lustful_Llama'))
-1 (Redditor(name='TheRedRebel4'), Redditor(name='LuckyCourse102'))
-1 (Redditor(name='TheRedRebel4'), Redditor(name='regarding_your_cat'))
-2 (Redditor(name='TheRedRebel4'), Redditor(name='Bombboy85'))
-9 (Redditor(name='PA562'), Redditor(name='RaddyMaddy'))
-6 (Redditor(name='PA562'), Redditor(name='Nafasion'))
-3 (Redditor(name='PA562'), Redditor(name='rotaercz'))
-3 (Redditor(name='LuckyCourse102'), Redditor(name='Bondominator'))
-6 (Redditor(name='LiveBeef'), Redditor(name='Brah-ma'))
-5 (Redditor(name='TimmyTurner3432'), Redditor(name='hebdomad7'))
-2 (Redditor(name='BackgroundSearch30'), Redditor(name='StephenElliott'))
-6 (Redditor(name='BackgroundS

In [75]:
giant_component.edges[e]

{'score': 477}

In [78]:
scores = [ giant_component.edges[e]['score'] for e in giant_component.edges]

In [84]:
min(scores)

-221